# *Run SCENIC+ with All PCW*

In [ ]:
import os
import scanpy as sc
sample = "ALL_pcw"
sample_cap = "ALL_PCW"
work_dir= "".join(["~/work/scp_", sample, "/"]) 
os.chdir(work_dir)

In [ ]:
!mkdir MalletRun_ALL_PCW

In [ ]:
import warnings
warnings.simplefilter(action='ignore')
import pycisTopic
pycisTopic.__version__

In [ ]:
from pycisTopic.cistopic_class import *

In [ ]:
count_matrix=sc.read_mtx("".join(["~/work/PeakMatrix/counts.mtx"]))
cellNames=pd.read_csv("".join(["~/work/PeakMatrix/cellNames.csv"]))["x"]
peakNames=pd.read_csv("".join(["~/work/PeakMatrix/peakNames.csv"]))["x"]

# Filter out lowly variable peaks

In [ ]:
from scipy.sparse import csr_matrix
df_sparse = csr_matrix(count_matrix.X)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

means = np.array(df_sparse.mean(axis=1)).flatten()
variances = np.array(df_sparse.power(2).mean(axis=1)).flatten() - means**2

plt.figure(figsize=(10, 6))
plt.scatter(means, variances, alpha=0.5, s=1)
plt.xlabel('Mean')
plt.ylabel('Variance')
plt.title('Mean-Variance Relationship')
plt.xscale('log')
plt.yscale('log')

valid_indices = means > 0
log_means = np.log(means[valid_indices])
log_variances = np.log(variances[valid_indices])

coefficients = np.polyfit(log_means, log_variances, deg=1)
polynomial = np.poly1d(coefficients)

fitted_variances = np.exp(polynomial(log_means))

plt.plot(means[valid_indices], fitted_variances, color='red', label='Fitted Line')
plt.legend()
plt.show()

mean_threshold = np.percentile(means, 30)
variance_threshold = np.percentile(variances, 30)

filtered_indices = np.where((means > mean_threshold) & (variances > variance_threshold))[0]

print(f"Filtered peak number: {len(filtered_indices)}")

In [ ]:
df_sparse_filtered = df_sparse[filtered_indices, :]
peakNames_filtered = pd.Series(peakNames.iloc[filtered_indices].values).reset_index(drop=True)

In [ ]:
# Create cisTopic object
cistopic_obj = create_cistopic_object(fragment_matrix=df_sparse_filtered, cell_names=cellNames, region_names=peakNames_filtered, tag_cells = False)

In [ ]:
# Adding cell information
cell_data =  pd.read_csv("".join(["~/work/FL_wnn_cellmeta.v01.csv"]), 
                         index_col=0)
cell_data.index = cell_data.index.str.replace('_', '#')
cistopic_obj.add_cell_data(cell_data)
cistopic_obj.cell_data['celltype']=cistopic_obj.cell_data['anno_wnn_v51']

In [ ]:
import gc
gc.collect()

In [ ]:
# Save cisTopic object
import pickle
pickle.dump(
    cistopic_obj,
    open("".join(["~/work/scp_", 
                  sample, 
                  "/cistopic_obj_",
                  sample_cap,
                  ".pkl"]), 
         "wb"))

# The following topic modeling can be two option

* With pycisTopic_polars_1xx version

In [ ]:
# # get binary matrix
# binary_matrix = cistopic_obj.binary_matrix
# # write to file
# mmwrite("binary_accessibility_matrix.mtx", binary_matrix)

# Then proceed with "pycisTopic_polars_1xx_CLI" folder

* Without pycisTopic_polars_1xx version

In [ ]:
# # Load cisTopic object
# import pickle
# with open("".join(["~/work/scp_", 
#                   sample, 
#                   "/cistopic_obj_",
#                   sample_cap,
#                   ".pkl"]), "rb") as file:
#     cistopic_obj = pickle.load(file)

In [ ]:
# print(cistopic_obj)

In [ ]:
# os.environ['MALLET_MEMORY'] = '700G'

In [ ]:
# from pycisTopic.lda_models import run_cgs_models_mallet
# # Configure path Mallet
# mallet_path="~/work/Mallet/bin/mallet"
# # Run models
# models=run_cgs_models_mallet(
#     cistopic_obj,
#     n_topics=[20,25,30,35,40,45],
#     n_cpu=20,
#     n_iter=150,
#     random_state=555,
#     alpha=50,
#     alpha_by_topic=True,
#     eta=0.1,
#     eta_by_topic=False,
#     tmp_path="".join(["MalletRun_", sample_cap, "/tmp/"]),
#     save_path="".join(["MalletRun_", sample_cap, "/"]),
#     mallet_path=mallet_path,
#     reuse_corpus=True
# )

In [ ]:
# # Load model list
# import pickle

# # Define the number of models and the file naming pattern
# num_topics = [15, 20, 25, 30, 35, 40]  # List of topic counts
# models = []  # Initialize an empty list to store the loaded models

# # Loop through each topic count and load the corresponding model
# for num in num_topics:
#     file_path = f"~/work/scp_{sample}/MalletRun_ALL_PCW/MalletRun_ALL_PCW.{num}_topics.model.pkl"
#     with open(file_path, "rb") as file:
#         model = pickle.load(file)  # Load the model
#         models.append(model)  # Append to the list

In [ ]:
# # Save models
# pickle.dump(
#     models,
#     open("".join(["~/work/scp_", 
#                   sample, 
#                   "/models_",
#                   sample_cap,
#                   ".pkl"]),
#          "wb"))

In [ ]:
# Load models
import pickle
with open("".join(["~/work/scp_", 
                  sample, 
                  "/models_",
                  sample_cap,
                  ".pkl"]), "rb") as file:
    models = pickle.load(file)

# Select Models

In [ ]:
from pycisTopic.lda_models import evaluate_models
model = evaluate_models(
    models,
    select_model = 35,
    return_model = True
)

In [ ]:
cistopic_obj.add_LDA_model(model)

In [ ]:
cistopic_obj.selected_model.region_topic

In [ ]:
# Save cisTopic object
import pickle
pickle.dump(
    cistopic_obj,
    open("".join(["~/work/scp_", 
                  sample, 
                  "/cistopic_obj_",
                  sample_cap,
                  "_withModel.pkl"]), 
         "wb"))

In [ ]:
# Load cisTopic object
import pickle
with open("".join(["~/work/scp_", 
                  sample, 
                  "/cistopic_obj_",
                  sample_cap,
                  "_withModel.pkl"]), "rb") as file:
    cistopic_obj = pickle.load(file)

# Do Clustering

In [ ]:
from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)

In [ ]:
find_clusters(
    cistopic_obj,
    target  = 'cell',
    k = 30,
    res = [1],
    prefix = 'pycisTopic_',
    scale = True,
    split_pattern = ''
)

In [ ]:
run_umap(
    cistopic_obj,
    target  = 'cell', scale=True)

In [ ]:
# Save cisTopic object
import pickle
pickle.dump(
    cistopic_obj,
    open("".join(["~/work/scp_", 
                  sample, 
                  "/cistopic_obj_",
                  sample_cap,
                  "_withUMAP.pkl"]), 
         "wb"))

In [ ]:
celltype_colors = {
    'HSC': '#E41A1C',
    'GP': '#E0FFFF',
    'Granulocyte': '#B3CDE3',
    'MEMP-t': '#E6AB02',
    'MEMP': '#FF7F00',
    'MEP': '#CD661D',
    'MEMP-Mast-Ery': '#FDCDAC',
    'MEMP-Ery': '#E9967A',
    'Early-Ery': '#CD5555',
    'Late-Ery': '#8B0000',
    'MEMP-MK': '#663C1F',
    'MK': '#40E0D0',
    'MastP-t': '#1E90FF',
    'MastP': '#1F78B4',
    'Mast': '#253494',
    'MDP': '#E6F5C9',
    'Monocyte': '#005A32',
    'Kupffer': '#00EE00',
    'cDC1': '#B3DE69',
    'cDC2': '#ADFF2F',
    'pDC': '#4DAF4A',
    'ASDC': '#CDC673',
    'LMP': '#FFF2AE',
    'LP': '#FFD92F',
    'Cycling-LP': '#FFFF33',
    'PreProB': '#FFF0F5',
    'ProB-1': '#FFB5C5',
    'ProB-2': '#E78AC3',
    'Large-PreB': '#CD1076',
    'Small-PreB': '#FF3E96',
    'IM-B': '#FF00FF',
    'NK': '#A020F0',
    'ILCP': '#49006A',
    'T': '#984EA3',
    'Hepatocyte': '#666666',
    'Endothelia': '#000000'
}

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke

plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['celltype'],
    target='cell', 
    num_columns=1,
    text_size=10,
    dot_size=1, 
    color_dictionary={'celltype': celltype_colors})

fig = plt.gcf()
ax = plt.gca()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel("UMAP_1", fontsize=12, labelpad=10)
ax.set_ylabel("UMAP_2", fontsize=12, labelpad=10)

ax.set_title("") 

ax.tick_params(axis='both', which='both', length=0)
ax.set_xticks([])
ax.set_yticks([])

for text in ax.texts:
    text.set_path_effects([withStroke(linewidth=0.7, foreground="black")])

plt.savefig("".join(["CellType_umap_", sample, ".pdf"]), dpi=300, bbox_inches='tight', format='pdf')

plt.close()

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 24))

plot_topic(
    cistopic_obj,
    reduction_name = 'UMAP',
    target = 'cell',
    num_columns=5
)

plt.savefig("".join(["Topic_umap_", sample, ".png"]), dpi=300, bbox_inches='tight')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

cell_topic_matrix = cistopic_obj.selected_model.cell_topic
cell_types = pd.Series(data=cistopic_obj.cell_data["celltype"], index=cell_topic_matrix.columns)

unique_cell_types = cell_types.unique()
cell_type_colors = cell_types.map(celltype_colors)

g = sns.clustermap(
    cell_topic_matrix,
    cmap="viridis",
    row_cluster=True,
    col_cluster=True,
    col_colors=cell_type_colors,
    figsize=(10, 10),
    cbar_kws={'label': 'Loading Value'},
    cbar_pos=(1, 0.2, 0.03, 0.6),
    dendrogram_ratio=(0.05, 0.05),
    # yticklabels=False,
    xticklabels=False
)

legend_handles = [
    plt.Line2D([0], [0], marker='o', color=color, markersize=10, linestyle='', label=label)
    for label, color in cell_type_lut.items()
]
legend = g.ax_col_colors.legend(
    handles=legend_handles,
    title="Cell Type",
    loc="upper left",
    bbox_to_anchor=(1.2, 1)
)
plt.setp(legend.get_texts(), fontsize='10')

g.fig.savefig("".join(["heatmap_cell_", sample, ".png"]), dpi=300, bbox_inches='tight')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

cell_topic_matrix = cistopic_obj.selected_model.cell_topic
cell_types = pd.Series(data=cistopic_obj.cell_data["celltype"], index=cell_topic_matrix.columns)

cell_topic_matrix.columns = cell_types

mean_topic_matrix = cell_topic_matrix.groupby(cell_topic_matrix.columns, axis=1).mean()

cell_type_colors = mean_topic_matrix.columns.map(celltype_colors)

g = sns.clustermap(
    mean_topic_matrix,
    cmap="viridis",
    row_cluster=True,
    col_cluster=True,
    col_colors=cell_type_colors,
    figsize=(7, 7),
    cbar_kws={'label': 'Average Loading'},
    cbar_pos=(1, 0.2, 0.03, 0.6),
    dendrogram_ratio=(0.05, 0.05),
    yticklabels=True,
    xticklabels=True
)

legend_handles = [
    plt.Line2D([0], [0], marker='o', color=color, markersize=10, linestyle='', label=label)
    for label, color in celltype_colors.items() if label in mean_topic_matrix.columns
]
legend = g.ax_col_colors.legend(
    handles=legend_handles,
    title="Cell Type",
    loc="upper left",
    bbox_to_anchor=(1.3, 0.85)
)
plt.setp(legend.get_texts(), fontsize='10')

plt.savefig("".join(["heatmap_celltype_", sample, ".pdf"]), dpi=300, bbox_inches='tight', format='pdf')

# Normalize scRNA-seq dara before Run SCENIC+

In [ ]:
import anndata as ad
adata = ad.read_h5ad("".join(["~/work/scrna_seurat_", 
                             sample_cap, ".h5ad"]))
print(adata)

In [ ]:
import scanpy as sc
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
adata.write("seurat_obj.h5ad")

# Get topic-based DARs

In [ ]:
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)
import numpy as np

imputed_acc_obj = impute_accessibility(
    cistopic_obj,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)

In [ ]:
# Load cisTopic object
import pickle
with open("".join(["~/work/scp_", 
                  sample, 
                  "/cistopic_obj_",
                  sample_cap,
                  "_withUMAP.pkl"]), "rb") as file:
    cistopic_obj = pickle.load(file)

In [ ]:
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)

variable_regions = find_highly_variable_features(
    imputed_acc_obj,
    min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    n_top_features=None,
    plot=True
)

In [ ]:
os.makedirs(os.path.join(work_dir, "".join(["region_sets_", sample]), "Topics_top_3k"), exist_ok = True)
from pycisTopic.topic_binarization import binarize_topics
region_bin_topics_top_3k = binarize_topics(
    cistopic_obj,
    method='ntop',
    plot=True,
    num_columns=5,
    ntop=3000
)

In [ ]:
from pycisTopic.utils import region_names_to_coordinates
for topic in region_bin_topics_top_3k:
    region_names_to_coordinates(
        region_bin_topics_top_3k[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(work_dir, "".join(["region_sets_", sample]), "Topics_top_3k", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )